In [7]:
#import libraries
import base64
import requests
import datetime
from urllib.parse import urlencode

In [8]:
#setting up variables
client_id = '5aedd1d0682f46618896dcea6108e3b4'
client_secret = 'e3c2c283017d452a93ddc951053196d2'
access_token = None

In [9]:
#https://www.youtube.com/watch?v=xdq6Gz33khQ&t=1318s
#code derived from link above
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_hasExpired = True
    c_id = None
    c_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret

    def encode_credentials (self):
        id = self.client_id
        secret = self.client_secret
        if secret == None or id == None:
            raise Exception("client id or secret is not set")
        client_creds = f"{id}:{secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def token_headers(self):
        c64_credentials = self.encode_credentials()
        return {
            "Authorization": f"Basic {c64_credentials}"
        }
    
    def token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_auth(self):
        r= requests.post(self.token_url, data=self.token_data(), headers=self.token_headers())
        if r.status_code in range(200,299):
            now = datetime.datetime.now()
            self.access_token = r.json()['access_token']
            expires = now + datetime.timedelta(seconds=r.json()['expires_in'])
            self.access_token_expires = expires
            self.access_token_hasExpired = expires < now
            print("success")
        print(r.status_code)

In [24]:
class SpotifyQuery():
    headers = None
    artist_lookup = None
    endpoint_url = "https://api.spotify.com/v1/search?"
    endpoint_query = None
    endpoint = None
    artist_row = []
    
    def __init__(self, access_token):
        self.headers = {
                "Authorization": f"Bearer {access_token}"
        }
    
    
    def get_artistid(self):
        artist_list = ReadArtists.getList()
        for i in artist_list:
            col = []
            artist_lookup = i
            endpoint_query = urlencode({"q": artist_lookup,"type": "artist", "limit": "1"})
            self.endpoint = self.endpoint_url+endpoint_query
            r = requests.get(self.endpoint, headers = self.headers)
            items = r.json()['artists']['items'][0]
            col.append(items['name'])
            col.append(items['id'])
            self.artist_row.append(col)
            print(items)
    
    #def get_albums(self):
     #   for i in artist_row:
      #      artist_id = i[1]
            
          
    

In [18]:
class ReadArtists():
    artist_list = None
    
    #reading all the artist from this list. all artist names will be held here
    def getList():
        file = open("artists.txt","r")
        raw_list = file.read()
        artist_list = raw_list.split(";")
        file.close()
        return artist_list
    
   # def writeList(filename):
    #    with open(filename, 'w') as f:
            
    
    

In [19]:
spotify = SpotifyAPI(client_id, client_secret)

In [20]:
spotify.get_auth()

success
200


In [25]:
access_token = spotify.access_token
query = SpotifyQuery(access_token)
query.get_artistid()
query.artist_row

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'followers': {'href': None, 'total': 67370935}, 'genres': ['canadian hip hop', 'canadian pop', 'hip hop', 'rap', 'toronto rap'], 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb4293385d324db8558179afd9', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab676161000051744293385d324db8558179afd9', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f1784293385d324db8558179afd9', 'width': 160}], 'name': 'Drake', 'popularity': 95, 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/4q3ewBCX7sLwd24euuV69X'}, 'followers': {'href': None, 'total': 56691222}, 'genres': ['reggaeton', 'trap latino'], 'href': 'https://api.spotify.com/v1/artists/4q3ewBCX7sLwd24euuV69X', 'id'

[['Drake', '3TVXtAsR1Inumwj472S9r4'], ['Bad Bunny', '4q3ewBCX7sLwd24euuV69X']]

In [16]:
ReadArtists.getList()

['Drake', 'Bad Bunny']